In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_train = pd.merge(pd.read_pickle('../data/genes-train.pk'),pd.read_csv('../data/training_variants.csv'),
               left_index=True,right_index=True)
df_test = pd.merge(pd.read_pickle('../data/genes-test.pk'),pd.read_csv('../data/test_variants.csv'),
                  left_index=True,right_index=True)
df_train.head()

,text,processed,ID,Gene,Variation,Class
0,Cyclin-dependent kinases (CDKs) regulate a var...,"[FAM58A, STAR, STAR, STAR, STAR, FAM58A, STAR,...",0,FAM58A,Truncating Mutations,1
1,Abstract Background Non-small cell lung canc...,"[CBL, CBL, EGFR, MET, CBL, CBL, MET, EGFR, KRA...",1,CBL,W802*,2
2,Abstract Background Non-small cell lung canc...,"[CBL, CBL, EGFR, MET, CBL, CBL, MET, EGFR, KRA...",2,CBL,Q249E,2
3,Recent evidence has demonstrated that acquired...,"[JAK2, JAK2, CBL, CBL, CBL, FLT3, CBL, V617F, ...",3,CBL,N454D,3
4,Oncogenic mutations in the monomeric Casitas B...,"[CBL, CBL, CBL, CBL, CBL, CBL, EGFR, CBL, CBL,...",4,CBL,L399V,4


In [3]:
text_series = pd.concat((df_train.processed,df_test.processed))

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
def return_same(x):
    return x
count_vectorizer = CountVectorizer(analyzer=return_same)
vecs = count_vectorizer.fit_transform(text_series)

In [5]:
count_vectorizer = CountVectorizer(analyzer=return_same)
vecs = count_vectorizer.fit_transform(text_series)
vecs_train = vecs[:df_train.shape[0]]
vecs_test = vecs[df_train.shape[0]:]

from sklearn.utils import shuffle
X,y = shuffle(vecs_train,df_train.Class)

lr_scores = cross_val_score(LogisticRegression(C=.01),X,y,scoring='neg_log_loss')
svm_scores = cross_val_score(SVC(C=.1,probability=True),X,y,n_jobs=-1,scoring='neg_log_loss')
xgb_scores = cross_val_score(XGBClassifier(n_estimators=500,learning_rate=.01,n_jobs=-1,subsample=.5)
                             ,X,y,scoring='neg_log_loss')
print('lr:  {:.3f} +/- {:.3f}'.format(lr_scores.mean(),lr_scores.std()))
print('svm: {:.3f} +/- {:.3f}'.format(svm_scores.mean(),svm_scores.std()))
print('xgb: {:.3f} +/- {:.3f}'.format(xgb_scores.mean(),xgb_scores.std()))

lr:  -1.478 +/- 0.028
svm: -1.347 +/- 0.014
xgb: -1.301 +/- 0.019


In [6]:
count_vectorizer = CountVectorizer(analyzer=return_same,min_df=10)
vecs = count_vectorizer.fit_transform(text_series)
vecs_train = vecs[:df_train.shape[0]]
vecs_test = vecs[df_train.shape[0]:]

X,y = shuffle(vecs_train,df_train.Class)

lr_scores = cross_val_score(LogisticRegression(C=.01),X,y,scoring='neg_log_loss')
svm_scores = cross_val_score(SVC(C=.1,probability=True),X,y,n_jobs=-1,scoring='neg_log_loss')
xgb_scores = cross_val_score(XGBClassifier(n_estimators=2000,learning_rate=.05,n_jobs=-1,subsample=.5,
                                           objective='multi:softprob',eval_metric='mlogloss')
                             ,X,y,scoring='neg_log_loss')
print('lr:  {:.3f} +/- {:.3f}'.format(lr_scores.mean(),lr_scores.std()))
print('svm: {:.3f} +/- {:.3f}'.format(svm_scores.mean(),svm_scores.std()))
print('xgb: {:.3f} +/- {:.3f}'.format(xgb_scores.mean(),xgb_scores.std()))

lr:  -1.422 +/- 0.054
svm: -1.363 +/- 0.013
xgb: -1.199 +/- 0.030


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
count_vectorizer = CountVectorizer(analyzer=return_same,min_df=10)
vecs = count_vectorizer.fit_transform(text_series)
vecs_train = vecs[:df_train.shape[0]]
vecs_test = vecs[df_train.shape[0]:]

X,y = shuffle(vecs_train,df_train.Class)

lr_scores = cross_val_score(LogisticRegression(C=.01),X,y,scoring='neg_log_loss')
svm_scores = cross_val_score(SVC(C=.1,probability=True),X,y,n_jobs=-1,scoring='neg_log_loss')
xgb_scores = cross_val_score(XGBClassifier(n_estimators=2000,learning_rate=.05,n_jobs=-1,subsample=.5,
                                           objective='multi:softprob',eval_metric='mlogloss')
                             ,X,y,scoring='neg_log_loss')
print('lr:  {:.3f} +/- {:.3f}'.format(lr_scores.mean(),lr_scores.std()))
print('svm: {:.3f} +/- {:.3f}'.format(svm_scores.mean(),svm_scores.std()))
print('xgb: {:.3f} +/- {:.3f}'.format(xgb_scores.mean(),xgb_scores.std()))

lr:  -1.429 +/- 0.073
svm: -1.376 +/- 0.016
xgb: -1.216 +/- 0.011


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer=return_same,min_df=10)
vecs = tfidf_vectorizer.fit_transform(text_series)
vecs_train = vecs[:df_train.shape[0]]
vecs_test = vecs[df_train.shape[0]:]

X,y = shuffle(vecs_train,df_train.Class)

lr_scores = cross_val_score(LogisticRegression(C=.01),X,y,scoring='neg_log_loss')
svm_scores = cross_val_score(SVC(C=.1,probability=True),X,y,n_jobs=-1,scoring='neg_log_loss')
xgb_scores = cross_val_score(XGBClassifier(n_estimators=2000,learning_rate=.05,n_jobs=-1,subsample=.5,
                                           objective='multi:softprob',eval_metric='mlogloss')
                             ,X,y,scoring='neg_log_loss')
print('lr:  {:.3f} +/- {:.3f}'.format(lr_scores.mean(),lr_scores.std()))
print('svm: {:.3f} +/- {:.3f}'.format(svm_scores.mean(),svm_scores.std()))
print('xgb: {:.3f} +/- {:.3f}'.format(xgb_scores.mean(),xgb_scores.std()))

lr:  -1.896 +/- 0.001
svm: -1.449 +/- 0.023
xgb: -1.243 +/- 0.004


In [9]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_topics=50, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0,n_jobs=-1)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer=return_same)
vecs = tfidf_vectorizer.fit_transform(text_series)
gene_vecs = lda.fit_transform(vecs)
gene_vecs_train = gene_vecs[:df_train.shape[0]]

X,y = shuffle(gene_vecs_train,df_train.Class)

lr_scores = cross_val_score(LogisticRegression(C=.01),X,y,scoring='neg_log_loss')
svm_scores = cross_val_score(SVC(C=.1,probability=True),X,y,n_jobs=-1,scoring='neg_log_loss')
xgb_scores = cross_val_score(XGBClassifier(n_estimators=2000,learning_rate=.05,n_jobs=-1,subsample=.5,
                                           objective='multi:softprob',eval_metric='mlogloss')
                             ,X,y,scoring='neg_log_loss')
print('lr:  {:.3f} +/- {:.3f}'.format(lr_scores.mean(),lr_scores.std()))
print('svm: {:.3f} +/- {:.3f}'.format(svm_scores.mean(),svm_scores.std()))
print('xgb: {:.3f} +/- {:.3f}'.format(xgb_scores.mean(),xgb_scores.std()))

lr:  -1.897 +/- 0.003
svm: -1.601 +/- 0.003
xgb: -1.415 +/- 0.051


In [11]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_topics=20, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0,n_jobs=-1)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer=return_same)
vecs = tfidf_vectorizer.fit_transform(text_series)
gene_vecs = lda.fit_transform(vecs)
gene_vecs_train = gene_vecs[:df_train.shape[0]]

X,y = shuffle(gene_vecs_train,df_train.Class)

lr_scores = cross_val_score(LogisticRegression(C=.01),X,y,scoring='neg_log_loss')
svm_scores = cross_val_score(SVC(C=.1,probability=True),X,y,n_jobs=-1,scoring='neg_log_loss')
xgb_scores = cross_val_score(XGBClassifier(n_estimators=2000,learning_rate=.05,n_jobs=-1,subsample=.5,
                                           objective='multi:softprob',eval_metric='mlogloss')
                             ,X,y,scoring='neg_log_loss')
print('lr:  {:.3f} +/- {:.3f}'.format(lr_scores.mean(),lr_scores.std()))
print('svm: {:.3f} +/- {:.3f}'.format(svm_scores.mean(),svm_scores.std()))
print('xgb: {:.3f} +/- {:.3f}'.format(xgb_scores.mean(),xgb_scores.std()))

lr:  -1.887 +/- 0.004
svm: -1.621 +/- 0.026
xgb: -1.457 +/- 0.046


In [12]:
df_train_stemmed = pd.merge(pd.read_pickle('../data/stem-train.pk'),pd.read_csv('../data/training_variants.csv'),
               left_index=True,right_index=True)
df_test_stemmed = pd.merge(pd.read_pickle('../data/stem-test.pk'),pd.read_csv('../data/test_variants.csv'),
                  left_index=True,right_index=True)
text_series_stem = pd.concat((df_train_stemmed.processed,df_test_stemmed.processed))
from sklearn.decomposition import LatentDirichletAllocation
del df_train['text']
del df_test['text']
del df_train['processed']
del df_test['processed']
del df_train_stemmed
del df_test_stemmed
lda_stem = LatentDirichletAllocation(n_topics=20, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0,n_jobs=-1)

tfidf_vectorizer_stem = TfidfVectorizer(analyzer=return_same,min_df=15,max_df=.8)
stem_vecs = tfidf_vectorizer_stem.fit_transform(text_series_stem)
print(stem_vecs.shape)
del text_series_stem
stem_vecs = lda_stem.fit_transform(stem_vecs)
stem_vecs_train = stem_vecs[:df_train.shape[0]]

X,y = shuffle(stem_vecs_train,df_train.Class)

lr_scores = cross_val_score(LogisticRegression(C=.01),X,y,scoring='neg_log_loss')
svm_scores = cross_val_score(SVC(C=.1,probability=True),X,y,n_jobs=-1,scoring='neg_log_loss')
xgb_scores = cross_val_score(XGBClassifier(n_estimators=2000,learning_rate=.05,n_jobs=-1,subsample=.5,
                                           objective='multi:softprob',eval_metric='mlogloss')
                             ,X,y,scoring='neg_log_loss')

print('lr:  {:.3f} +/- {:.3f}'.format(lr_scores.mean(),lr_scores.std()))
print('svm: {:.3f} +/- {:.3f}'.format(svm_scores.mean(),svm_scores.std()))
print('xgb: {:.3f} +/- {:.3f}'.format(xgb_scores.mean(),xgb_scores.std()))

(8989, 48026)
lr:  -1.917 +/- 0.001
svm: -1.925 +/- 0.067
xgb: -1.561 +/- 0.057


In [ ]:
df_train_stemmed = pd.merge(pd.read_pickle('../data/stem-train.pk'),pd.read_csv('../data/training_variants.csv'),
               left_index=True,right_index=True)
df_test_stemmed = pd.merge(pd.read_pickle('../data/stem-test.pk'),pd.read_csv('../data/test_variants.csv'),
                  left_index=True,right_index=True)
text_series_stem = pd.concat((df_train_stemmed.processed,df_test_stemmed.processed))
from sklearn.decomposition import LatentDirichletAllocation
del df_train_stemmed
del df_test_stemmed
lda_stem = LatentDirichletAllocation(n_topics=50, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0,n_jobs=-1)

tfidf_vectorizer_stem = TfidfVectorizer(analyzer=return_same,min_df=15,max_df=.8)
stem_vecs = tfidf_vectorizer_stem.fit_transform(text_series_stem)
print(stem_vecs.shape)
del text_series_stem
stem_vecs = lda_stem.fit_transform(stem_vecs)
stem_vecs_train = stem_vecs[:df_train.shape[0]]

X,y = shuffle(stem_vecs_train,df_train.Class)

lr_scores = cross_val_score(LogisticRegression(C=.01),X,y,scoring='neg_log_loss')
svm_scores = cross_val_score(SVC(C=.1,probability=True),X,y,n_jobs=-1,scoring='neg_log_loss')
xgb_scores = cross_val_score(XGBClassifier(n_estimators=2000,learning_rate=.05,n_jobs=-1,subsample=.5,
                                           objective='multi:softprob',eval_metric='mlogloss')
                             ,X,y,scoring='neg_log_loss')
print('lr:  {:.3f} +/- {:.3f}'.format(lr_scores.mean(),lr_scores.std()))
print('svm: {:.3f} +/- {:.3f}'.format(svm_scores.mean(),svm_scores.std()))
print('xgb: {:.3f} +/- {:.3f}'.format(xgb_scores.mean(),xgb_scores.std()))

(8989, 48026)
lr:  -1.917 +/- 0.001
svm: -1.998 +/- 0.040
xgb: -1.571 +/- 0.053


In [ ]:
df_train_stemmed = pd.merge(pd.read_pickle('../data/stem-train.pk'),pd.read_csv('../data/training_variants.csv'),
               left_index=True,right_index=True)
df_test_stemmed = pd.merge(pd.read_pickle('../data/stem-test.pk'),pd.read_csv('../data/test_variants.csv'),
                  left_index=True,right_index=True)
text_series_stem = pd.concat((df_train_stemmed.processed,df_test_stemmed.processed))
from sklearn.decomposition import LatentDirichletAllocation
del df_train_stemmed
del df_test_stemmed
lda_stem = LatentDirichletAllocation(n_topics=500, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0,n_jobs=-1)

tfidf_vectorizer_stem = TfidfVectorizer(analyzer=return_same,min_df=15,max_df=.8)
stem_vecs = tfidf_vectorizer_stem.fit_transform(text_series_stem)
print(stem_vecs.shape)
del text_series_stem
stem_vecs = lda_stem.fit_transform(stem_vecs)
stem_vecs_train = stem_vecs[:df_train.shape[0]]

X,y = shuffle(stem_vecs_train,df_train.Class)

lr_scores = cross_val_score(LogisticRegression(C=.01),X,y,scoring='neg_log_loss')
svm_scores = cross_val_score(SVC(C=.1,probability=True),X,y,n_jobs=-1,scoring='neg_log_loss')
xgb_scores = cross_val_score(XGBClassifier(n_estimators=2000,learning_rate=.05,n_jobs=-1,subsample=.5,
                                           objective='multi:softprob',eval_metric='mlogloss')
                             ,X,y,scoring='neg_log_loss')
print('lr:  {:.3f} +/- {:.3f}'.format(lr_scores.mean(),lr_scores.std()))
print('svm: {:.3f} +/- {:.3f}'.format(svm_scores.mean(),svm_scores.std()))
print('xgb: {:.3f} +/- {:.3f}'.format(xgb_scores.mean(),xgb_scores.std()))

In [ ]:
sg_vecs_train = np.hstack((gene_vecs_train, stem_vecs_train))

In [ ]:
sg_vecs_train.shape

(3321, 70)

In [ ]:
X,y = shuffle(sg_vecs_train,df_train.Class)

lr_scores = cross_val_score(LogisticRegression(C=.01),X,y,scoring='neg_log_loss')
svm_scores = cross_val_score(SVC(C=.1,probability=True),X,y,n_jobs=-1,scoring='neg_log_loss')
xgb_scores = cross_val_score(XGBClassifier(n_estimators=2000,learning_rate=.05,n_jobs=-1,subsample=.5,
                                           objective='multi:softprob',eval_metric='mlogloss')
                             ,X,y,scoring='neg_log_loss')
print('lr:  {:.3f} +/- {:.3f}'.format(lr_scores.mean(),lr_scores.std()))
print('svm: {:.3f} +/- {:.3f}'.format(svm_scores.mean(),svm_scores.std()))
print('xgb: {:.3f} +/- {:.3f}'.format(xgb_scores.mean(),xgb_scores.std()))

lr:  -1.847 +/- 0.002
svm: -1.622 +/- 0.015
xgb: -1.448 +/- 0.040
